In [1]:
import torch
import numpy as np
from tqdm import tqdm
from os.path import join
import matplotlib.pyplot as plt
from torch.nn.functional import pairwise_distance
from torch.utils.data import DataLoader, Dataset,random_split

import config_gcp as config
from model import SiameseNetwork,training
from utils import imshow, show_plot, load_dataset,decision_stub


training_dir = config.training_dir
testing_dir = config.testing_dir
training_csv = config.training_csv
testing_csv = config.testing_csv

In [ ]:
# Load the dataset
siamese_dataset = load_dataset(training_dir,training_csv)

# Load the weight to the network
net = SiameseNetwork().cuda()
state_dict = "AdamW  Parameter Group 0 amsgrad-False betas-0.9, 0.999 eps-1e-08 lr-0.0001 weight_decay-0.0005  batch_size-32validation_error0.36272189349112427.pth"
#"AdamW  Parameter Group 0 amsgrad-False betas-0.9, 0.999 eps-1e-08 lr-1e-05 weight_decay-0.0005  batch_size-32validation_error0.4443007972419737.pth"
net.load_state_dict(torch.load(join("state_dict",state_dict)))
net.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Create test dataloader
num_test = 5000 # select number of samples to be inferenced
num_other = siamese_dataset.__len__()-num_test
siamese_test, _ = random_split(siamese_dataset, [num_test,num_other])

test_dataloader = DataLoader(
    siamese_test, shuffle=True, num_workers=8, batch_size=1
)

In [6]:
# Evaluate the Eucledean distance between each pair
test_distance = np.zeros((test_dataloader.__len__(),2))
for i, data in enumerate(tqdm(test_dataloader), 0):
    x0, x1, label = data
    output1, output2 = net(x0.to(device), x1.to(device))
    eucledian_distance = pairwise_distance(output1, output2)
    test_distance[i,0] = eucledian_distance.detach().cpu().numpy()
    test_distance[i,1] = label.detach().cpu().numpy()*2-1
decision_stub(test_distance.tolist(),verbose=True)

100%|██████████████████████████████████████| 5000/5000 [00:37<00:00, 133.39it/s]

j_star = 0
theta_star = 0.002477
polorization = -1
Empirical Error = 0.220000



0.22